# <h1 style="text-align:center;">Análise de Curto Prazo da Inflação Utilizando Painel Dinâmico</h1>

## <h2 style="text-align:left;">Introdução</h2>

<p>
Este notebook tem como objetivo realizar a coleta, organização e análise de dados de inflação com foco em uma observação de curto prazo. Utilizando modelos de painel dinâmico, o estudo busca compreender a evolução dos índices de preços ao consumidor (IPCA) e suas categorias específicas, considerando também variáveis macroeconômicas relevantes como crédito, taxa de câmbio, taxa de juros e preços internacionais de commodities.
</p>

<p>
A análise segue uma abordagem sistemática, com as seguintes etapas:
</p>

<ol>
    <li>Coleta de dados de diferentes fontes, como APIs públicas e bases de dados econômicos.</li>
    <li>Transformação e organização dos dados em um formato adequado para modelos de painel.</li>
    <li>Aplicação de modelos GMM (Generalized Method of Moments) para estimar a dinâmica da inflação.</li>
    <li>Previsões dos próximos valores da inflação com base no modelo ajustado.</li>
</ol>

<p>
A modelagem utiliza o pacote <strong>pydynpd</strong>, que oferece ferramentas para lidar com endogeneidade e capturar relações temporais nos dados.
</p>

<p>
A seguir, apresentamos as etapas do estudo, detalhando a coleta dos dados e a implementação dos modelos de painel dinâmico para gerar insights sobre a inflação de curto prazo.
</p>


In [2]:
import requests
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from pydynpd import regression

## <h2 style="text-align:left;">Coleta de Dados</h2>

<p>
Nesta seção, foram implementadas funções para a coleta de dados de séries temporais econômicas relevantes. Os dados são obtidos a partir de duas fontes principais:
</p>

<ul>
    <li>
        <strong>Banco Central do Brasil (API BCB):</strong> Foi utilizada uma função genérica para coletar dados de diferentes séries temporais econômicas, como crédito, taxa de câmbio e taxa de juros.
    </li>
    <li>
        <strong>Yahoo Finance:</strong> Foi implementada uma função específica para coletar dados do preço do petróleo Brent, uma variável importante para capturar as flutuações de preços de commodities.
    </li>
</ul>

<p>
Essas funções retornam os dados em formato de <code>DataFrame</code>, já organizados com nomes apropriados e prontos para integração com outras etapas do processo analítico.
</p>

<p>
O código a seguir define as duas funções principais para a coleta de dados:
</p>


In [3]:
def fetch_bcb_data(series_dict, start_date, end_date):
    """
    Função genérica para buscar séries temporais da API do Banco Central.

    Args:
        series_dict (dict): Dicionário com IDs das séries temporais como chave e nomes como valor.
        start_date (str): Data inicial no formato 'dd/mm/yyyy'.
        end_date (str): Data final no formato 'dd/mm/yyyy'.

    Returns:
        DataFrame: DataFrame com as séries buscadas e suas respectivas datas.
    """
    base_url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs."
    all_data = pd.DataFrame()

    for series_id, series_name in series_dict.items():
        url = f"{base_url}{series_id}/dados?formato=json&dataInicial={start_date}&dataFinal={end_date}"
        response = requests.get(url)

        if response.status_code == 200:
            data = pd.DataFrame(response.json())
            data['data'] = pd.to_datetime(data['data'], format='%d/%m/%Y')
            data.rename(columns={'valor': series_name}, inplace=True)

            if all_data.empty:
                all_data = data
            else:
                all_data = all_data.merge(data, on='data', how='outer')
        else:
            print(f"Erro ao buscar a série {series_id}: {response.status_code}")

    return all_data

def fetch_brent_prices(start_date, end_date):
    """
    Função para buscar os preços históricos do petróleo Brent do Yahoo Finance.

    Args:
        start_date (str): Data inicial no formato 'YYYY-MM-DD'.
        end_date (str): Data final no formato 'YYYY-MM-DD'.

    Returns:
        DataFrame: DataFrame com as datas e preços de fechamento do Brent.
    """
    ticker = "BZ=F"  # Código do Brent no Yahoo Finance
    brent_data = yf.download(ticker, start=start_date, end=end_date)

    if not brent_data.empty:
        brent_data = brent_data[['Close']].reset_index()
        brent_data.rename(columns={'Close': 'tx_brent'}, inplace=True)
        return brent_data
    else:
        print("Nenhum dado encontrado para o Brent no período especificado.")
        return pd.DataFrame()

## <h2 style="text-align:left;">Configuração dos Parâmetros e Séries Temporais</h2>

<p>
Nesta etapa, foi definido o intervalo de datas para a coleta de dados e configurados os dicionários que mapeiam os códigos das séries econômicas às suas descrições. Os códigos são específicos para cada fonte de dados e permitem identificar as variáveis relevantes para a análise.
</p>

<p>
As séries temporais incluídas na análise são:
</p>

<ul>
    <li><strong>Índices de Inflação (IPCA):</strong> Dados detalhados por categoria, como alimentos, habitação, transporte, entre outros.</li>
    <li><strong>CAGED:</strong> Número de empregos formais registrado no Cadastro Geral de Empregados e Desempregados.</li>
    <li><strong>Saldo de Crédito:</strong> Dados de crédito para pessoas jurídicas e físicas.</li>
    <li><strong>Taxa de Câmbio:</strong> Cotação do dólar comercial.</li>
    <li><strong>Taxa Selic:</strong> Taxa básica de juros do Brasil.</li>
</ul>

<p>
A configuração utiliza dicionários que associam os códigos de cada série ao nome da variável, facilitando a coleta e organização posterior dos dados.
</p>

<p>
O código abaixo ilustra como esses parâmetros e séries são configurados:
</p>


In [122]:
# Definir o intervalo de datas
data_inicial = "01/01/2013"  # Alterar conforme necessário
data_final = datetime.today().strftime('%d/%m/%Y')
start_date_brent = "2013-01-01"
end_date_brent = datetime.today().strftime('%Y-%m-%d')


# Dicionários com os códigos e nomes das séries
indices_inflacao = {
    1635: "ipca_alimentos", 
    1636: "ipca_habitacao", 
    1637: "ipca_residencia", 
    1638: "ipca_vestuario", 
    1639: "ipca_transportes", 
    1640: "ipca_comunicacao", 
    1641: "ipca_saude", 
    1642: "ipca_despesas", 
    1643: "ipca_educacao", 
    4447: "ipca_comercializaveis", 
    4448: "ipca_nao_comercializaveis"
}

caged = {
    28763: "caged_empregos"
}

saldo_credito = {
    20540: "cred_pj", 
    20541: "cred_pf"
}

taxa_cambio = {
    1: "tx_cambio"
}

taxa_selic = {
    11: "tx_selic"
}

## <h2 style="text-align:left;">Coleta dos Dados de Inflação e Regressores</h2>

<p>
Com os parâmetros e séries temporais definidos, iniciamos o processo de coleta dos dados. Nesta etapa, utilizamos as funções previamente implementadas para buscar as informações necessárias de cada grupo:
</p>

<ul>
    <li><strong>Regressandos:</strong> Dados dos índices de inflação (IPCA) coletados da API do Banco Central.</li>
    <li><strong>Regressores:</strong> Variáveis macroeconômicas, como empregos formais, crédito, taxa de câmbio, taxa Selic e preços do petróleo Brent.</li>
</ul>

<p>
Após a coleta, os dados são convertidos para o tipo <code>datetime</code> no formato UTC para garantir consistência temporal durante as análises subsequentes. 
</p>

<p>
Abaixo, o código realiza a coleta dos dados para os dois grupos e exibe os resultados iniciais:
</p>


In [123]:
# Buscar dados para cada grupo
print("Buscando dados dos índices de inflação...")
regressandos = fetch_bcb_data(indices_inflacao, data_inicial, data_final)
regressandos["data"] = pd.to_datetime(regressandos["data"],utc=True)  # Garantir que seja do tipo datetime utc
regressandos[regressandos.columns.drop("data")] = regressandos[regressandos.columns.drop("data")].apply(pd.to_numeric, errors='coerce')

display(regressandos)
display(regressandos.describe())

print("Buscando dados dos regressores...")

# Buscar preços do petróleo Brent
regressores_dicts = {**caged, **saldo_credito, **taxa_cambio, **taxa_selic}
regressores = fetch_bcb_data(regressores_dicts, data_inicial, data_final)
regressores[regressores.columns.drop("data")] = regressores[regressores.columns.drop("data")].apply(pd.to_numeric, errors='coerce')

display(regressores)
display(regressores.describe())

Buscando dados dos índices de inflação...


,data,ipca_alimentos,ipca_habitacao,ipca_residencia,ipca_vestuario,ipca_transportes,ipca_comunicacao,ipca_saude,ipca_despesas,ipca_educacao,ipca_comercializaveis,ipca_nao_comercializaveis
0,2013-01-01 00:00:00+00:00,1.99,-0.20,1.15,-0.53,0.75,-0.08,0.73,1.55,0.35,1.03,1.35
1,2013-02-01 00:00:00+00:00,1.45,-2.38,0.53,0.55,0.81,0.10,0.65,0.57,5.40,0.63,1.58
2,2013-03-01 00:00:00+00:00,1.14,0.51,0.11,0.15,-0.09,0.13,0.32,0.54,0.56,0.36,0.69
3,2013-04-01 00:00:00+00:00,0.96,0.62,0.63,0.65,-0.19,-0.32,1.28,0.61,0.10,0.24,0.90
4,2013-05-01 00:00:00+00:00,0.31,0.75,0.46,0.84,-0.25,0.08,0.94,0.41,0.06,0.24,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...
138,2024-07-01 00:00:00+00:00,-1.00,0.77,0.48,-0.02,1.82,0.18,0.22,0.52,0.08,0.14,0.13
139,2024-08-01 00:00:00+00:00,-0.44,-0.51,0.74,0.39,0.00,0.10,0.25,0.25,0.73,0.38,-0.25
140,2024-09-01 00:00:00+00:00,0.50,1.80,-0.19,0.18,0.14,-0.05,0.46,-0.31,0.05,0.59,-0.01
141,2024-10-01 00:00:00+00:00,1.06,1.49,0.43,0.37,-0.38,0.52,0.38,0.70,0.04,0.78,0.31


,ipca_alimentos,ipca_habitacao,ipca_residencia,ipca_vestuario,ipca_transportes,ipca_comunicacao,ipca_saude,ipca_despesas,ipca_educacao,ipca_comercializaveis,ipca_nao_comercializaveis
count,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,0.583636,0.508671,0.340350,0.356154,0.431119,0.107063,0.534056,0.473427,0.536853,0.454056,0.457622
std,0.769614,0.911159,0.548457,0.612411,1.076352,0.478814,0.448305,0.378745,1.412312,0.485616,0.436266
min,-1.070000,-2.380000,-1.370000,-1.150000,-4.510000,-2.080000,-0.710000,-0.310000,-3.470000,-0.760000,-0.530000
25%,0.050000,0.170000,-0.050000,-0.045000,-0.085000,-0.025000,0.335000,0.220000,0.050000,0.110000,0.170000
50%,0.560000,0.510000,0.350000,0.350000,0.400000,0.080000,0.480000,0.400000,0.110000,0.420000,0.450000
75%,1.045000,0.810000,0.660000,0.660000,0.930000,0.235000,0.730000,0.615000,0.265000,0.715000,0.685000
max,3.380000,5.290000,1.820000,2.110000,3.810000,2.090000,2.330000,1.720000,6.280000,1.940000,1.640000


Buscando dados dos regressores...


,data,caged_empregos,cred_pj,cred_pf,tx_cambio,tx_selic
0,2013-01-01,39655697.0,1277890.0,1088275.0,NaN,NaN
1,2013-01-02,NaN,NaN,NaN,2.0415,0.027260
2,2013-01-03,NaN,NaN,NaN,2.0464,0.027260
3,2013-01-04,NaN,NaN,NaN,2.0425,0.027260
4,2013-01-07,NaN,NaN,NaN,2.0312,0.027260
...,...,...,...,...,...,...
3065,2024-12-20,NaN,NaN,NaN,6.0780,0.045513
3066,2024-12-23,NaN,NaN,NaN,6.1612,0.045513
3067,2024-12-24,NaN,NaN,NaN,6.1541,0.045513
3068,2024-12-26,NaN,NaN,NaN,6.1656,0.045513


,data,caged_empregos,cred_pj,cred_pf,tx_cambio,tx_selic
count,3070,1.420000e+02,1.430000e+02,1.430000e+02,3012.000000,3011.000000
mean,2018-12-26 18:24:37.524429824,4.102208e+07,1.697981e+06,2.096868e+06,4.017151,0.035377
min,2013-01-01 00:00:00,3.805776e+07,1.277890e+06,1.088275e+06,1.952800,0.007469
25%,2015-12-23 06:00:00,3.903027e+07,1.440153e+06,1.520281e+06,3.166625,0.024620
50%,2018-12-27 12:00:00,4.036383e+07,1.591349e+06,1.803027e+06,3.898800,0.039270
75%,2021-12-27 18:00:00,4.204232e+07,1.936732e+06,2.687180e+06,5.147525,0.048159
max,2024-12-27 00:00:00,4.763475e+07,2.417755e+06,3.897222e+06,6.199100,0.052531
std,NaN,2.562373e+06,3.075667e+05,7.953485e+05,1.155991,0.013821


## <h2 style="text-align:left;">Tratamento e Integração do Preço do Petróleo (Brent)</h2>

<p>
Nesta etapa, foi realizado o tratamento dos dados do preço do petróleo Brent, coletados a partir do Yahoo Finance. Os dados são preparados e integrados com o conjunto de variáveis macroeconômicas já coletadas (regressores). As principais etapas incluem:
</p>

<ul>
    <li><strong>Tratamento dos Dados do Brent:</strong> Ajuste no índice temporal, renaming das colunas e formatação da coluna de data para que esteja formatada corretamente.</li>
    <li><strong>Criação de Subconjunto:</strong> Seleção das colunas relevantes, como o preço do Brent (<code>prc_brent</code>) e a data.</li>
    <li><strong>Integração com os Regressores:</strong> Merging dos dados do Brent com o DataFrame de regressores, garantindo consistência na coluna <code>data</code>.</li>
</ul>

<p>
Caso a coluna <code>data</code> não esteja presente em ambos os DataFrames, uma mensagem de erro é exibida para indicar inconsistências nos dados.
</p>

<p>
O código abaixo realiza o tratamento e a integração dos dados:
</p>


In [124]:
brent_prices = fetch_brent_prices(start_date_brent, end_date_brent)

if not brent_prices.empty:
    # # Resetar índice e garantir que a coluna 'data' seja consistente
    brent_prices = brent_prices.reset_index(drop=True) # Remove qualquer MultiIndex
    brent_prices = brent_prices.set_index('Date')
    brent_prices.columns = ['tx_brent']    
    brent_prices = brent_prices.rename(columns={"tx_brent": "prc_brent"})  
    brent_prices["data"] = pd.to_datetime(brent_prices.index, utc=True)  # Garantir que seja do tipo datetime
    regressores["data"] = pd.to_datetime(regressores["data"], utc=True)  # Garantir que seja do tipo datetime
    
    brent_prices = brent_prices.reset_index(drop=True) # Remove qualquer MultiIndex
    brent_new = brent_prices[['prc_brent', 'data']]
    display(brent_new)
    

    # Verificar se ambas as tabelas têm a coluna 'data'
    if "data" in regressores.columns and "data" in brent_new.columns:
        # Mesclar com os regressores
        regressores_merged = regressores.merge(brent_new, on="data", how="outer")
        display(regressores_merged)
    else:
        print("Erro: A coluna 'data' não está presente em ambos os DataFrames.")

[*********************100%***********************]  1 of 1 completed


,prc_brent,data
0,112.470001,2013-01-02 00:00:00+00:00
1,112.139999,2013-01-03 00:00:00+00:00
2,111.309998,2013-01-04 00:00:00+00:00
3,111.400002,2013-01-07 00:00:00+00:00
4,111.940002,2013-01-08 00:00:00+00:00
...,...,...
3000,72.879997,2024-12-19 00:00:00+00:00
3001,72.940002,2024-12-20 00:00:00+00:00
3002,72.629997,2024-12-23 00:00:00+00:00
3003,73.580002,2024-12-24 00:00:00+00:00


,data,caged_empregos,cred_pj,cred_pf,tx_cambio,tx_selic,prc_brent
0,2013-01-01 00:00:00+00:00,39655697.0,1277890.0,1088275.0,NaN,NaN,NaN
1,2013-01-02 00:00:00+00:00,NaN,NaN,NaN,2.0415,0.027260,112.470001
2,2013-01-03 00:00:00+00:00,NaN,NaN,NaN,2.0464,0.027260,112.139999
3,2013-01-04 00:00:00+00:00,NaN,NaN,NaN,2.0425,0.027260,111.309998
4,2013-01-07 00:00:00+00:00,NaN,NaN,NaN,2.0312,0.027260,111.400002
...,...,...,...,...,...,...,...
3137,2024-12-20 00:00:00+00:00,NaN,NaN,NaN,6.0780,0.045513,72.940002
3138,2024-12-23 00:00:00+00:00,NaN,NaN,NaN,6.1612,0.045513,72.629997
3139,2024-12-24 00:00:00+00:00,NaN,NaN,NaN,6.1541,0.045513,73.580002
3140,2024-12-26 00:00:00+00:00,NaN,NaN,NaN,6.1656,0.045513,73.260002


## <h2 style="text-align:left;">Preenchimento de Valores Nulos e Exportação dos Dados</h2>

<p>
Após a integração dos dados do preço do petróleo Brent com os regressores, foi realizado o preenchimento dos valores nulos e garantimos o alinhamento das datas entre os conjuntos de dados. As etapas incluem:
</p>

<ul>
    <li><strong>Preenchimento de Valores Nulos:</strong> Utilizando o método <code>forward fill</code> (<code>ffill</code>) para preencher valores ausentes com o último valor válido disponível, garantindo a continuidade das séries temporais.</li>
    <li><strong>Alinhamento Temporal:</strong> Os dados de regressandos e regressores são filtrados para manter apenas as datas comuns, com base na presença da variável <code>caged_empregos</code>.</li>
    <li><strong>Exportação dos Dados:</strong> Os DataFrames finais para regressandos e regressores são salvos no formato Parquet, permitindo armazenamento eficiente e compatibilidade com etapas analíticas subsequentes.</li>
</ul>

<p>
O código abaixo executa essas etapas e conclui o processamento dos dados:
</p>


In [ ]:
# Preencher valores nulos com o método forward fill
regressores_merged = regressores_merged.fillna(method='ffill') # Alinha colunas diárias com as mensais

# Garantir alinhamento pelas datas do CAGED e dos Índices de Inflação
if "caged_empregos" in regressores_merged.columns:
    common_dates = regressandos["data"].dropna().unique()
    regressandos = regressandos[regressandos["data"].isin(common_dates)]
    regressores_merged = regressores_merged[regressores_merged["data"].isin(common_dates)]

# Exportar para arquivos Parquet
regressandos.to_parquet("regressandos.parquet", index=False)
regressores_merged.to_parquet("regressores.parquet", index=False)

print("Exportação concluída: regressandos e regressores salvos como arquivos Parquet.")


## <h2 style="text-align:left;">Transformação e Mesclagem dos Dados</h2>

<p>
Nesta etapa é feita a leitura dos arquivos Parquet previamente exportados e as transformações adicionais para preparar os dados para análise. As principais ações incluem:
</p>

<ul>
    <li><strong>Leitura dos Arquivos Parquet:</strong> Os conjuntos de dados de regressandos e regressores são carregados e suas estruturas são exibidas para validação.</li>
    <li><strong>Transformação Logarítmica:</strong> Em algumas variáveis econômicas (como <code>caged_empregos</code>, <code>cred_pj</code>, <code>cred_pf</code> e <code>prc_brent</code>) para capturar variações proporcionais, reduzindo efeitos de escala e melhorando a interpretação dos resultados.</li>
    <li><strong>Mesclagem dos Dados:</strong> Os DataFrames de regressandos e regressores são combinados pela coluna <code>data</code>, resultando em um conjunto integrado.</li>
    <li><strong>Transformação para Painel:</strong> As colunas de inflação são transformadas em uma estrutura de painel utilizando <code>pd.melt()</code>, com cada índice de IPCA representando um "indivíduo" no painel.</li>
</ul>

<p>
O DataFrame final é exportado no formato Parquet e exibido para validação. A estrutura resultante está pronta para análise dinâmica e modelagem de painel.
</p>

<p>
O código abaixo implementa essas etapas:
</p>


In [126]:
# Ler os arquivos Parquet e realizar o merge
regressandos_df = pd.read_parquet("regressandos.parquet")
regressores_df = pd.read_parquet("regressores.parquet")

display(regressores_df.dtypes)

# Transformar a variáveis logaritmo natural para capturar variação
regressores_df["caged_empregos"] = np.log(regressores_df["caged_empregos"].astype(float))
regressores_df["cred_pj"] = np.log(regressores_df["cred_pj"].astype(float))
regressores_df["cred_pf"] = np.log(regressores_df["cred_pf"].astype(float))
regressores_df["prc_brent"] = np.log(regressores_df["prc_brent"].astype(float))

# Merge dos DataFrames pela data
merged_df = pd.merge(regressandos_df, regressores_df, on="data", how="outer")

# Exibir as primeiras linhas do DataFrame resultante
print("DataFrame mesclado:")

# Opcional: Exportar o DataFrame mesclado
merged_df.to_parquet("merged_data.parquet", index=False)
print("DataFrame mesclado exportado como 'merged_data.parquet'.")

# Transformar colunas de inflação em indivíduos usando pd.melt()
merged_df = pd.melt(
    merged_df,
    id_vars= regressores_df.columns,  # Colunas que permanecerão como estão
    value_vars=regressandos_df.columns,  # Colunas que serão transformadas em valores
    var_name="tipo_ipca",  # Nova coluna para os nomes das variáveis de IPCA
    value_name="tx_ipca"  # Nova coluna para os valores das variáveis de IPCA
)

# Transformando a taxa de inflação em decimal
merged_df["tx_ipca"] = merged_df["tx_ipca"] / 100

# Exibir o DataFrame resultante
print("DataFrame após transformação com pd.melt():")
display(merged_df)

data              datetime64[ns, UTC]
caged_empregos                float64
cred_pj                       float64
cred_pf                       float64
tx_cambio                     float64
tx_selic                      float64
prc_brent                     float64
dtype: object

DataFrame mesclado:
DataFrame mesclado exportado como 'merged_data.parquet'.
DataFrame após transformação com pd.melt():


,data,caged_empregos,cred_pj,cred_pf,tx_cambio,tx_selic,prc_brent,tipo_ipca,tx_ipca
0,2013-01-01 00:00:00+00:00,17.495745,14.060721,13.900104,NaN,NaN,NaN,ipca_alimentos,0.0199
1,2013-02-01 00:00:00+00:00,17.499994,14.069239,13.906059,1.9843,0.027260,4.760121,ipca_alimentos,0.0145
2,2013-03-01 00:00:00+00:00,17.504579,14.092420,13.918081,1.9848,0.027371,4.704110,ipca_alimentos,0.0114
3,2013-04-01 00:00:00+00:00,17.510963,14.096395,13.933157,2.0186,0.027445,4.710251,ipca_alimentos,0.0096
4,2013-05-01 00:00:00+00:00,17.513722,14.110647,13.950027,2.0017,0.028333,4.604670,ipca_alimentos,0.0031
...,...,...,...,...,...,...,...,...,...
1568,2024-07-01 00:00:00+00:00,17.665900,14.657588,15.135687,5.5893,0.039270,4.461300,ipca_nao_comercializaveis,0.0013
1569,2024-08-01 00:00:00+00:00,17.670973,14.667337,15.146596,5.6681,0.039270,4.376009,ipca_nao_comercializaveis,-0.0025
1570,2024-09-01 00:00:00+00:00,17.676283,14.684447,15.157375,5.6562,0.039270,4.366913,ipca_nao_comercializaveis,-0.0001
1571,2024-10-01 00:00:00+00:00,17.679073,14.684538,15.165407,5.4521,0.040168,4.298101,ipca_nao_comercializaveis,0.0031


## <h2 style="text-align:left;">Interpretação dos Resultados do Modelo e Previsão</h2>

<p>
Nesta seção, foram feitas duas funções principais para análise e aplicação dos resultados do modelo GMM dinâmico:
</p>

### <h3 style="text-align:left;">1. Interpretação dos Testes do Modelo</h3>
<p>
A função <code>interpret_test_results</code> avalia a validade do modelo ajustado através dos seguintes testes estatísticos:
</p>
<ul>
    <li><strong>Teste de Hansen:</strong> Verifica a validade dos instrumentos utilizados no modelo. Um p-valor maior que 0,05 indica que as restrições de superidentificação não são rejeitadas, sugerindo que os instrumentos são válidos.</li>
    <li><strong>Testes de Arellano-Bond (AR):</strong> Avaliam a correlação serial dos erros em diferentes ordens:
        <ul>
            <li><strong>AR(1):</strong> Correlação de primeira ordem nas diferenças; é esperado que esteja presente em modelos GMM.</li>
            <li><strong>AR(2):</strong> Correlação de segunda ordem; sua ausência sugere que o modelo está corretamente especificado.</li>
        </ul>
    </li>
</ul>
<p>
Os resultados desses testes fornecem informações sobre a especificação do modelo e a validade dos instrumentos.
</p>

### <h3 style="text-align:left;">2. Previsão dos Próximos Valores</h3>
<p>
A função <code>forecasting</code> utiliza os coeficientes estimados pelo modelo GMM para prever os próximos valores da variável dependente. As etapas incluem:
</p>
<ul>
    <li><strong>Extração de Coeficientes:</strong> Os coeficientes do modelo são filtrados para identificar os relevantes para as variáveis independentes e defasadas.</li>
    <li><strong>Cálculo dos Valores Previstos:</strong> Para cada período futuro, os valores das variáveis independentes e defasadas são multiplicados pelos coeficientes correspondentes para calcular a previsão.</li>
    <li><strong>Construção do DataFrame:</strong> Os valores previstos são adicionados ao DataFrame original, criando um conjunto completo para os períodos futuros.</li>
</ul>
<p>
Essa abordagem permite estimar a trajetória futura da inflação com base nas relações dinâmicas capturadas pelo modelo GMM.
</p>

<p>
O código abaixo implementa a interpretação dos testes e a geração de previsões:
</p>


In [127]:
def interpret_test_results(gmm_model: regression.abond):
    """
    Interpreta os resultados dos testes de Hansen e Arellano-Bond de um modelo GMM ajustado com PyDynPD.
    
    Args:
        gmm_model: Objeto dynamic_panel_model do PyDynPD.
    """
    gmm_model = gmm_model.models[0]  # Acessar o modelo GMM ajustado

    print("-" * 100)
    # Teste de Hansen
    if hasattr(gmm_model, "hansen"):
        hansen_test = gmm_model.hansen  # Objeto do teste de Hansen
        hansen_stat = getattr(hansen_test, "test_value", None)  # Estatística do teste
        hansen_pval = getattr(hansen_test, "p_value", None)  # p-valor do teste
        hansen_df = getattr(hansen_test, "df", None)  # Graus de liberdade
        # print(f"Hansen Test: Chi({hansen_df}) = {hansen_stat:.3f}, p-value = {hansen_pval:.3f}")
        if hansen_pval is not None and hansen_pval > 0.05:
            print("Teste de Hansen: As restrições de superidentificação não são rejeitadas. Os instrumentos são válidos.")
        else:
            print("Teste de Hansen: As restrições de superidentificação são rejeitadas. Pode haver problemas de validade nos instrumentos.")
        
    # Testes de Arellano-Bond
    if hasattr(gmm_model, "AR_list"):
        for index, ar_test in enumerate(gmm_model.AR_list, start=1):
            ar_order = index  # Ordem do teste AR
            z_stat = getattr(ar_test, "AR", None)  # Estatística z
            p_value = getattr(ar_test, "P_value", None)  # p-valor
            # print(f"Arellano-Bond Test AR({ar_order}): z = {z_stat:.3f}, p-value = {p_value:.3f}")
            if ar_order == 1:
                if p_value < 0.05:
                    print("AR(1): Evidência de correlação serial de primeira ordem nas diferenças.")
                else:
                    print("AR(1): Sem evidência de correlação serial de primeira ordem.")
            elif ar_order == 2:
                if p_value > 0.05:
                    print("AR(2): Sem evidência de correlação serial de segunda ordem nas diferenças. Modelo parece válido.")
                else:
                    print("AR(2): Evidência de correlação serial de segunda ordem. Modelo pode estar mal especificado.")
    print("-" * 100)


def forecasting(df, sys_gmm, dep_var, independent_vars):
    """
    Calcula o valor previsto de inflação para o próximo período com base nos coeficientes do modelo GMM.

    Args:
        df (pd.DataFrame): DataFrame com os dados históricos, incluindo as variáveis independentes.
        sys_gmm (regression.abond): Objeto GMM com os resultados da regressão.
        dep_var (str): Nome da variável dependente (ex.: "tx_ipca").
        independent_vars (list): Lista de variáveis explicativas usadas no modelo.

    Returns:
        pd.DataFrame: DataFrame contendo o valor previsto para o próximo período.
    """
    # Obter coeficientes e estatísticas z do modelo
    regression_table = sys_gmm.models[0].regression_table

    # Adicionar as variáveis independentes com defasagens e a variável dependente defasada
    lagged_vars = independent_vars + [f"L1.{dep_var}"] + [f"L1.{var}" for var in independent_vars]

    # Filtrar coeficientes significativos (|z| >= 1.96)
    coefficients = {
        row["variable"]: row["coefficient"]
        for _, row in regression_table.iterrows()
        if abs(row["z_value"]) > 1.96 and row["variable"] in lagged_vars
    }

    # Exibir as variáveis utilizadas com base no critério Z
    print("Variáveis utilizadas com base no critério Z (|z| >= 1.96) e intercessão com variáveis defasadas:")
    for var in coefficients.keys():
        print(f"- {var}")

    # Verificar se há coeficientes significativos suficientes
    if not coefficients:
        raise ValueError("Nenhum coeficiente significativo encontrado para previsão.")

    # Adicionar a variável defasada (L1) ao DataFrame
    df[f"L1.{dep_var}"] = df[dep_var].shift(1)
    for var in independent_vars:
        df[f"L1.{var}"] = df[var].shift(1)

    # Criar uma nova linha de previsão
    forecast_row = {"data": df["data"].max() + pd.DateOffset(months=1)}

    # Preencher valores baseados nos coeficientes e variáveis explicativas
    for var in lagged_vars:
        if var in coefficients:
            forecast_row[var] = df[var].iloc[-1] if not pd.isna(df[var].iloc[-1]) else 0.0

    # Calcular previsão para o próximo período
    forecast_value = 0
    print("Cálculo da previsão para o próximo período:")
    for var, coef in coefficients.items():
        if var in forecast_row:
            print(f"  {var}: {forecast_row[var]} * {coef}")
            forecast_value += coef * forecast_row[var]

    forecast_row[f"{dep_var}_forecast"] = forecast_value

    # Retornar apenas a previsão para o próximo período
    return pd.DataFrame([forecast_row])[["data", f"{dep_var}_forecast"]]

## <h2 style="text-align:left;">Preparação Final dos Dados e Configuração do Modelo GMM</h2>

<p>
Nesta etapa, foram realizados os ajustes finais nos dados e configurados os parâmetros necessários para a aplicação do modelo GMM (Generalized Method of Moments). As ações principais incluem:
</p>

### <h3 style="text-align:left;">1. Ajustes nos Dados</h3>
<ul>
    <li><strong>Remoção de Categorias Específicas do IPCA:</strong> As categorias <code>ipca_comercializaveis</code> e <code>ipca_nao_comercializaveis</code> são excluídas do conjunto de dados para focar nas outras categorias de inflação.</li>
    <li><strong>Definição das Variáveis Dependentes:</strong> O IPCA (<code>tx_ipca</code>) é definido como a variável dependente do modelo.</li>
    <li><strong>Seleção das Variáveis Independentes:</strong> Variáveis macroeconômicas como <code>caged_empregos</code>, <code>cred_pj</code>, <code>cred_pf</code>, <code>tx_cambio</code>, <code>tx_selic</code> e <code>prc_brent</code> são incluídas como explicativas.</li>
</ul>

### <h3 style="text-align:left;">2. Configuração do Modelo</h3>
<ul>
    <li><strong>Instrumentos:</strong> Os instrumentos GMM são configurados para incluir defasagens das variáveis dependentes e independentes, enquanto instrumentos externos são especificados para as variáveis exógenas.</li>
    <li><strong>Opções do Modelo:</strong> As opções incluem a inclusão de dummies de tempo, colapso dos instrumentos e uso de transformações <em>forward orthogonal deviations</em> (FOD), que preservam a ortogonalidade dos erros.</li>
</ul>

### <h3 style="text-align:left;">3. Adição de Defasagens</h3>
<p>
Defasagem de 1 período é adicionada às variáveis dependentes e explicativas. Isso permite capturar as dinâmicas temporais do modelo e preparar os dados para o GMM.
</p>

<p>
O código abaixo implementa essas ações, preparando os dados para a análise de painel dinâmico:
</p>


In [128]:
# Retirar ipca de comercializaveis e não comercializaveis
merged_df = merged_df[merged_df["tipo_ipca"] != "ipca_comercializaveis"]
merged_df = merged_df[merged_df["tipo_ipca"] != "ipca_nao_comercializaveis"]

# Variáveis dependentes (IPCA)
dependent_vars = [
    "tx_ipca"
]

# Variáveis explicativas (defasadas e contemporâneas)
independent_vars = [
    "caged_empregos", "cred_pj", "cred_pf", "tx_cambio", "tx_selic", "prc_brent"
]

# Adicionar defasagens de todas as variáveis
lagged_vars = dependent_vars + independent_vars

instruments_template = [
    f"gmm({''.join(dependent_vars)}, 2:9)",
    f"iv({' '.join(independent_vars)})"
]

# Opções da regressão
options = [
    "timedumm",  # Inclui dummies de tempo no modelo. 
                 # Útil para capturar efeitos específicos de períodos e evitar viés temporal.

    "collapse",  # Colapsa os instrumentos GMM, reduzindo a dimensionalidade do conjunto de instrumentos.
                 # Isso ajuda a evitar sobreidentificação, melhorando os resultados do teste de Hansen.

    "onestep",   # Realiza a regressão GMM em uma única etapa (one-step GMM).
                 # Normalmente usada em modelos iniciais; menos robusta a heterocedasticidade.

    # "iterated", # (Opcional) Ativa o método de GMM iterado, ajustando os pesos a cada iteração.
                 # Produz estimativas mais eficientes em algumas circunstâncias, mas pode aumentar o tempo de execução.

    # "hqic",      # Calcula e utiliza o Critério de Informação de Hannan-Quinn (HQIC) para avaliação do modelo.
                 # HQIC penaliza modelos com muitos instrumentos, favorecendo especificações mais simples.

    "fod",       # Usa a transformação "forward orthogonal deviations" em vez de diferenças em primeira ordem (first differences).
                 # Essa transformação preserva a ortogonalidade do erro, melhorando a eficiência do modelo.

    # "nolevel"    # Exclui a transformação em nível, utilizando apenas a equação em diferenças.
                 # Isso é característico de Difference GMM. Deve ser removido se você quiser ativar System GMM.
]


# Criar defasagens no DataFrame
for var in lagged_vars:
    for lag in range(1, 3):  # Adicionando 2 defasagens (1:2)
        merged_df[f"L{lag}.{var}"] = merged_df[var].shift(lag)



## <h2 style="text-align:left;">Aplicação do Modelo GMM e Geração de Previsões</h2>

<p>
Nesta etapa, aplica-se o modelo GMM para cada variável dependente (IPCA) e analisa-se os resultados utilizando os instrumentos e opções configurados. Apesar de o GMM apresentar desafios metodológicos em painéis longos (T >=  N), medidas específicas foram adotadas para endereçar os possíveis problemas. Adicionalmente, foi gerada a previsão para o próximo mês como análise complementar. As principais ações incluem:
</p>

### <h3 style="text-align:left;">1. Configuração do Modelo para Cada Índice IPCA</h3>
<ul>
    <li><strong>Formatação de Instrumentos:</strong> Os instrumentos GMM e externos são configurados de acordo com o índice IPCA atual.</li>
    <li><strong>Montagem do Comando:</strong> O comando é construído dinamicamente para incluir a variável dependente, variáveis explicativas, instrumentos e opções de configuração do modelo.</li>
</ul>

### <h3 style="text-align:left;">2. Execução da Regressão</h3>
<ul>
    <li><strong>Estimativa do Modelo:</strong> Foi utilizada a função <code>regression.abond</code> do <code>pydynpd</code> para ajustar o modelo de painel dinâmico.</li>
    <li><strong>Validação e Mitigação:</strong> Estratégias como o uso de instrumentos colapsados e validação por meio dos testes de Hansen e Arellano-Bond foram adotadas para tratar possíveis problemas de sobreidentificação e correlação serial nos resíduos.</li>
    <li><strong>Interpretação dos Resultados:</strong> A função <code>interpret_test_results</code> foi usada para avaliar a validade do modelo e verificar a robustez dos instrumentos.</li>
</ul>

### <h3 style="text-align:left;">3. Geração de Previsões</h3>
<ul>
    <li><strong>Coeficientes:</strong> Como os diferentes sub-índices de inflação foram tratados como indivíduos no painel, os coeficientes dos regressores representam o efeito médio que impacta todos os sub-índices igualmente. Dessa forma, o ruído de dos diferentes sub-índices está sendo capturado pelo resíduo idiossincrático, característico do painel com efeitos fixos.</li>
    <li><strong>Projeções:</strong> Este estudo não tem o objetivo de realizar a previsão do índice, apenas de enteder a magnutide do efeito das variáveis utilizadas. Todavia, a função de previsão do próximo mês (step) foi feita e pode ser utilizada com algum cuidado em relação à sua interpretação: o modelo retorna um valor expurgando todos os efeitos individuais dos diferentes sub-índices que compõem o índice agregado.</li>
    <li><strong>Resultados:</strong> A previsão é exibida para avaliação e entendimento da tendência projetada.</li>
</ul>

<p>
O código abaixo realiza essas etapas, aplicando o modelo GMM para cada índice IPCA e gerando a previsão baseadas nos coeficientes estimados (e estatísticamente significativos):
</p>



In [134]:


# Implementação do System GMM
for dep_var in dependent_vars:
    # Formatar as variáveis explicativas e instrumentos para a dependente atual
    formatted_instruments = []
    for var in independent_vars:
        formatted_instruments = instruments_template

    regressors =  independent_vars + [f"L1.{dep_var}"] + [f"L{lag}.{var}" for var in independent_vars for lag in range(1, 2)]

    cmd_str = (
        dep_var + " " +
        " ".join(regressors) + " | " +
        " ".join(formatted_instruments) + " | " +
        " ".join(options)
    )

    print("Regression: \n" + "\n".join([part for part in cmd_str.split("|")]))
 
    print(f"Executando regressão para variável dependente: {dep_var}")
    sys_gmm = regression.abond(cmd_str, merged_df, ['tipo_ipca', "data"])
    interpret_test_results(sys_gmm)

    # Fazer previsão para o próximo mês (só faz sentido para ipca isolado, objetivo aqui é entender o que afeta todos os indices)
    forecast = forecasting(
        df=merged_df,
        sys_gmm=sys_gmm,
        dep_var=dep_var,
        independent_vars=independent_vars
    )

    # Exibir previsões
    print(f"Previsões para {dep_var}:")
    display(forecast)
print("Portanto, espera-se uma inflação de {0:.2f}% para o mês de {1}".format(
                                forecast["tx_ipca_forecast"].values[0] * 100,
                                pd.to_datetime(forecast["data"].values[0]).strftime('%B, %Y')
))

Regression: 
tx_ipca caged_empregos cred_pj cred_pf tx_cambio tx_selic prc_brent L1.tx_ipca L1.caged_empregos L1.cred_pj L1.cred_pf L1.tx_cambio L1.tx_selic L1.prc_brent 
 gmm(tx_ipca, 2:9) iv(caged_empregos cred_pj cred_pf tx_cambio tx_selic prc_brent) 
 timedumm collapse onestep fod
Executando regressão para variável dependente: tx_ipca
 Dynamic panel-data estimation, one-step system GMM
 Group variable: tipo_ipca                         Number of obs = 1260      
 Time variable: data                               Min obs per group: 140    
 Number of instruments = 158                       Max obs per group: 142    
 Number of groups = 9                              Avg obs per group: 140.00 
+--------------------------------+------------+---------------------+------------+-----------+-----+
|            tx_ipca             |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+--------------------------------+------------+---------------------+------------+-----------

,data,tx_ipca_forecast
0,2024-12-01 00:00:00+00:00,0.010353


Portanto, espera-se uma inflação de 1.04% para o mês de December, 2024
